In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
alumnosRDD = [
    (79850,'7503', 9, '2019-03-07'), # (padrón, materia, nota,fecha YYYY-MM-DD)
    (83159,'7503', 4, '2019-12-07'),
    (79850,'7503', 9, '2019-03-07'),
    (75890,'7503', 9, '2019-07-07'),
    (83159,'8150', 1, '2017-02-10'),
    (89500,'8350', 2, '2018-01-11'),
    (89500,'7708', 5, '2017-12-28'),
    (95898,'7502', 7, '2016-02-19'),
    (92345,'9205', 8, '2016-05-02'),
    (92345,'9305', 4, '2018-01-07')   
]

alumnosRDD2 = [
    (79850,'Fulano', 'Fabian'), # (padrón, apellido, nombre)
    (83159,'Mengano','Miguel'),
    (75890,'Fernandez','Federico'),
    (89500,'Lopez','Carlos'),
    (95898,'Martinez','Martin'),
    (92345,'Sultano','Santiago')  
]


In [3]:
data = sc.parallelize(alumnosRDD);
data0 = sc.parallelize(alumnosRDD2)

In [4]:
aprobadosUltimos2anios = data.filter(lambda x: (x[2] >= 4 and x[3] >='2018-01-01'))

In [5]:
# Respuesta pregunta 1:
aprobadosUltimos2anios.collect()

[(79850, '7503', 9, '2019-03-07'),
 (83159, '7503', 4, '2019-12-07'),
 (79850, '7503', 9, '2019-03-07'),
 (75890, '7503', 9, '2019-07-07'),
 (92345, '9305', 4, '2018-01-07')]

In [6]:
#Ahora necesitaria armar un RDD con el padron y la nota para poder hacer el promedio
data2=data.map(lambda x: (x[0], x[2]))
data2.collect()

[(79850, 9),
 (83159, 4),
 (79850, 9),
 (75890, 9),
 (83159, 1),
 (89500, 2),
 (89500, 5),
 (95898, 7),
 (92345, 8),
 (92345, 4)]

In [7]:
#Sumo todas las notas , luego necesitaria dividir por la cantidad de materias o notas que posee cada alumno
data3=data2.reduceByKey(lambda x, y: x+y)
data3.collect()

[(89500, 7), (92345, 12), (79850, 18), (75890, 9), (95898, 7), (83159, 5)]

In [8]:
#Genero entonces un RDD con padron , un 1 y la nota para luego cuando sume las notas por padron contar sumando el 1 cuantas
#notas sume por padron
data4=data.map(lambda x: (x[0], (1, x[2])))

In [9]:
data4.collect()

[(79850, (1, 9)),
 (83159, (1, 4)),
 (79850, (1, 9)),
 (75890, (1, 9)),
 (83159, (1, 1)),
 (89500, (1, 2)),
 (89500, (1, 5)),
 (95898, (1, 7)),
 (92345, (1, 8)),
 (92345, (1, 4))]

In [10]:
data5=data4.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))
data5.collect()

[(89500, (2, 7)),
 (92345, (2, 12)),
 (79850, (2, 18)),
 (75890, (1, 9)),
 (95898, (1, 7)),
 (83159, (2, 5))]

In [11]:
data5.map(lambda x: (x[0], x[1][1]/x[1][0])).collect()

[(89500, 3.5),
 (92345, 6.0),
 (79850, 9.0),
 (75890, 9.0),
 (95898, 7.0),
 (83159, 2.5)]

In [14]:
#En un solo paso seria asi (rta pregunta2)
data6=data4.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0], x[1][1]/x[1][0])).collect()
data6

[(89500, 3.5),
 (92345, 6.0),
 (79850, 9.0),
 (75890, 9.0),
 (95898, 7.0),
 (83159, 2.5)]

In [137]:
def getValue(item):   
    return item[1]
resultadoFinal=sorted(data6, key=getValue)
padronPromedio=resultadoFinal[-1]
padronPromedio

(79850, 9.0)

In [77]:
#Otra forma de obtener el resultado anterior
result=data4.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0], x[1][1]/x[1][0])).takeOrdered(1)
print(result)

[(75890, 9.0)]


In [15]:
#O sino esta otra forma
data7=data4.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0], x[1][1]/x[1][0])).reduce(lambda x, y: x if x[1] > y[1] else y)
data7

(75890, 9.0)

In [135]:
data0.filter(lambda x:(x[0] == padronPromedio[0])).collect()

[(79850, 'Fulano', 'Fabian')]

In [126]:
data0.join(data).collect()

[(92345, ('Sultano', '9205')),
 (92345, ('Sultano', '9305')),
 (79850, ('Fulano', '7503')),
 (79850, ('Fulano', '7503')),
 (75890, ('Fernandez', '7503')),
 (95898, ('Martinez', '7502')),
 (89500, ('Lopez', '8350')),
 (89500, ('Lopez', '7708')),
 (83159, ('Mengano', '7503')),
 (83159, ('Mengano', '8150'))]